<a href="https://colab.research.google.com/github/Anechka2212/Machine-learning-technologies/blob/master/lab2_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install pandasql

  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
import numpy as np
import pandas as pd
import pandasql as ps
from time import time

In [22]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
data_diagnostic = pd.read_csv(url, sep=',',header=None)
data_diagnostic.columns = ['D' + str(i) for i in range(32)]
data_diagnostic.head()

,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [23]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data'
data_prognostic = pd.read_csv(url, sep=',',header=None)
data_prognostic.columns = ['P' + str(i) for i in range(35)]
data_prognostic.head()

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


In [24]:
# 1 - diagnosis, 2 - radius, 22 - worst radius
t0_diagnostic = time()
aggregations_diagnostic = {
    'D1':'sum',
    'D22': lambda x: max(x)
}
print(data_diagnostic.groupby('D2').agg(aggregations_diagnostic))
t1_diagnostic = time()
print('It (diagnostic) takes: ', t1_diagnostic - t0_diagnostic)

       D1     D22
D2               
6.981   B   7.930
7.691   B   8.678
7.729   B   9.077
7.760   B   9.456
8.196   B   8.964
8.219   B   9.092
8.571   B   9.473
8.597   B   8.952
8.598   B   9.565
8.618   B   9.507
8.671   B   9.262
8.726   B   9.628
8.734   B  10.170
8.878   B   9.981
8.888   B   9.733
8.950   B   9.414
9.000   B   9.699
9.029   B  10.310
9.042   B  10.060
9.173   B  10.010
9.268   B  10.280
9.295   B  10.570
9.333   B   9.845
9.397   B   9.965
9.405   B  10.850
9.423   B  10.490
9.436   B  12.020
9.465   B  10.410
9.504   B  10.230
9.567   B  10.510
...    ..     ...
20.570  M  24.990
20.580  M  23.240
20.590  M  23.860
20.600  M  25.740
20.640  M  25.370
20.730  M  32.490
20.920  M  24.290
20.940  M  25.580
21.090  M  26.680
21.100  M  25.680
21.160  M  29.170
21.370  M  22.690
21.560  M  25.450
21.610  M  26.230
21.710  M  30.750
21.750  M  28.190
22.010  M  27.660
22.270  M  28.400
23.090  M  30.790
23.210  M  31.010
23.270  M  28.010
23.290  M  25.120
23.510  M 

In [25]:
# 1 - diagnosis, 2 - radius, 22 - worst radius
t0_diagnostic = time()
aggr_query_diagnostic = '''
    SELECT 
        sum('D1'),
        'D2'
    FROM data_diagnostic 
    GROUP BY 'D2'
    '''
print(ps.sqldf(aggr_query_diagnostic, locals()).set_index('D22'))
t1_diagnostic = time()
print('It (diagnostic) takes: ', t1_diagnostic - t0_diagnostic)

KeyError: ignored